In [ ]:
!pip install -U transformers accelerate ipywidgets

# Colab Einstellungen ändern

Wir benötigen für diese Übung eine GPU

im Menü -> Laufzeit -> Laufzeittyp ändern -> T4 GPU -> Speichern

![image](images/colab-gpu-1.png)

![image](images/colab-gpu-2.png)

# OpenSource LLMs vom Huggingface-Hub verwenden

https://huggingface.co/models?pipeline_tag=text-generation&sort=trending

In [ ]:
from transformers import pipeline

In [ ]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

## Pipelines

Die transformers pipelines sind die einfachste Möglichkeit Modelle zu verwenden. Kurz gesagt stellen die Pipelines sicher, dass das Preprocessing (Tokenizen, Chat Template, etc ...) richtig angewendet wird.

In [ ]:
pipe = pipeline(
    "conversational",
    model=model_name,
    torch_dtype="auto",
    device_map="auto",
    max_new_tokens=512,
)

In [ ]:
messages = [{'role': 'user', 'content': 'Kennst du ein Donauwellenrezept?'}]

In [ ]:
pipe(messages)

## Huggingface AutoModel-Klassen

Etwas "aufwändiger" ist die Verwendung der AutoModel/AutoTokenizer Klassen. Wir müssen sicherstellen, dass wir:
- Modell und Tokenizer laden
- die Messages tokenizen
- das Chat-Template verwenden
- alle Daten auf dem richtigen und gleichen "Device" (CPU oder GPU) sind
- den erzeugten Text (Tokens) decoden - also die Tokens in Text umwandeln

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
# Modell und Tokenizer erstellen
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Nachrichten tokenizen, und zusammen mit dem Modell auf das gleiche "device" legen
messages = [{'role': 'user', 'content': 'Kennst du ein Donauwellenrezept?'}]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

device = 'cuda:0'
model_inputs = encodeds.to(device)
model.to(device)

In [ ]:
# Erzeugen der eigentlichen Ausgabe
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

In [ ]:
# Ausgabe von Tokens -> Text umwandeln und ausgeben
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])